<a href="https://colab.research.google.com/github/ddoo0922/Dacon_Nerdiness/blob/main/ExtraTreeClassifier%2BExtreTreeRegressor%2BLGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 

from sklearn.impute import KNNImputer


from string import ascii_lowercase
from itertools import combinations
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import  GradientBoostingClassifier
import lightgbm as lgb

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
#Data Load
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# Q

In [ ]:
train.head()

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0


In [ ]:
Answers = []
for i in range(1, 27):
    Answers.append('Q' + str(i))
    
Q_Answers = []
for i in range(1, 21):
    Q_Answers.append('Q' + str(i))

In [ ]:
def knull(col):
    imputer = KNNImputer(n_neighbors=3)
    a = imputer.fit_transform(train[col])
    train[col] = a

In [ ]:
knull(Answers)

In [ ]:
# Test data의 결측치 train의 최빈값으로 채우기
for i in range(len(Answers)):
    test[Answers[i]]=test[Answers[i]].fillna(train[Answers[i]].mode()[0])

In [ ]:
# 1~20번 항목 더해서 Q_score, 21~26번 항목 더해서 Qs_score 생성
Q_secret= ['Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26']

dataset = [train, test]

for data in dataset:
    data['Q_score'] = data[Q_Answers].sum(axis=1)
    data['Qs_score'] = data[Q_secret].sum(axis=1)

In [ ]:
# 마키아벨리즘 테스트(1~20번 항목)에서 T, V, M 컬럼 생성
for data in dataset:
    data['T'] = data['Q1']+data['Q2']+data['Q3']+data['Q7']+data['Q10']+data['Q12']+data['Q15']+data['Q16'] 
    data['V'] = data['Q4']+data['Q5']+data['Q8']+data['Q11']+data['Q13']+data['Q20']
    data['M'] = data['Q9']+data['Q19']

In [ ]:
for data in dataset:
    print(data['M'])

0         6.0
1         4.0
2         9.0
3         7.0
4         7.0
         ... 
14995     5.0
14996     5.0
14997     8.0
14998    10.0
14999     9.0
Name: M, Length: 15000, dtype: float64
0         8.0
1        10.0
2         7.0
3         9.0
4         6.0
         ... 
35447     8.0
35448    10.0
35449    10.0
35450    10.0
35451     9.0
Name: M, Length: 35452, dtype: float64


In [ ]:
# 마키아벨리즘 스코어, 분산 피처 생성
for data in dataset:
    data['Mach_score'] = data[Answers].mean(axis = 1)
    data['Mach_var'] = data[Answers].var(axis = 1)

In [ ]:
# 마키아벨리즘 비율 피처 생성
Ancoms = list(combinations(Answers, 2))
for data in dataset:
    for a,b in Ancoms:
        data['mach_%s_dv%s'%(a,b)] = data[a]/data[b]

C:\Users\thdal\AppData\Local\Temp/ipykernel_28880/4282676114.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data['mach_%s_dv%s'%(a,b)] = data[a]/data[b]


In [ ]:
# Q drop
for data in dataset:
    data.drop([('Q'+str(i) )for i in range(1,27)], axis = 1, inplace = True)

In [ ]:
train.head()

,index,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,...,mach_Q22_dvQ23,mach_Q22_dvQ24,mach_Q22_dvQ25,mach_Q22_dvQ26,mach_Q23_dvQ24,mach_Q23_dvQ25,mach_Q23_dvQ26,mach_Q24_dvQ25,mach_Q24_dvQ26,mach_Q25_dvQ26
0,0,USA,3,553,6,4.0,3.0,5.0,1.0,3.0,...,5.000000,1.000000,5.0,5.0,0.200000,1.00,1.00,5.0,5.0,1.0
1,1,USA,5,85,120,4.0,2.0,3.0,5.0,3.0,...,0.200000,0.333333,0.5,0.2,1.666667,2.50,1.00,1.5,0.6,0.4
2,2,NLD,9,108,100,1.0,2.0,3.0,1.0,5.0,...,0.500000,0.250000,0.5,0.2,0.500000,1.00,0.40,2.0,0.8,0.4
3,3,USA,2,121,139,3.0,3.0,3.0,4.0,5.0,...,1.333333,1.000000,1.0,2.0,0.750000,0.75,1.50,1.0,2.0,2.0
4,4,ITA,3,640,216,3.0,3.0,4.0,4.0,4.0,...,0.666667,0.500000,0.5,0.5,0.750000,0.75,0.75,1.0,1.0,1.0


# TIPI

In [ ]:
TIPI = []
for i in range(1,11):
    TIPI.append('TIPI'+str(i))

In [ ]:
knull(TIPI)

In [ ]:
train['TIPI10']

0        3.0
1        2.0
2        2.0
3        3.0
4        2.0
        ... 
14995    3.0
14996    2.0
14997    1.0
14998    1.0
14999    2.0
Name: TIPI10, Length: 15000, dtype: float64

In [ ]:
for i in range(len(TIPI)):
    test[TIPI[i]]=test[TIPI[i]].fillna(train[TIPI[i]].mode()[0])

In [ ]:
# tp1~10 항목 사용해 Ex, Ag, Con, Es, Op 컬럼 만들기

train['Extraverted'] = train['TIPI1'] - train['TIPI6']
train['Warm'] = train['TIPI7']-train['TIPI2']
train['Dependable'] = train['TIPI3']-train['TIPI8']
train['Calm'] = train['TIPI9']-train['TIPI4']
train['OpenMind'] = train['TIPI5']-train['TIPI10']

test['Extraverted'] = test['TIPI1'] - test['TIPI6']
test['Warm'] = test['TIPI7']-test['TIPI2']
test['Dependable'] = test['TIPI3']-test['TIPI8']
test['Calm'] = test['TIPI9']-test['TIPI4']
test['OpenMind'] = test['TIPI5']-test['TIPI10']

In [ ]:
# tp 비율 피처 생성
tpcoms = list(combinations(TIPI, 2))

for data in dataset:
    for a,b in tpcoms:
        data['%s_dv_%s'%(a,b)] = data[a]/data[b]

In [ ]:
train.drop([('TIPI'+str(i)) for i in range(1,10)], axis=1, inplace = True)
train.drop('TIPI10', axis = 1, inplace = True)

test.drop([('TIPI'+str(i)) for i in range(1,10)], axis=1, inplace = True)
test.drop('TIPI10', axis = 1, inplace = True)

In [ ]:
train.head()

,index,country,introelapse,testelapse,surveyelapse,VCL1,VCL2,VCL3,VCL4,VCL5,...,TIPI6_dv_TIPI7,TIPI6_dv_TIPI8,TIPI6_dv_TIPI9,TIPI6_dv_TIPI10,TIPI7_dv_TIPI8,TIPI7_dv_TIPI9,TIPI7_dv_TIPI10,TIPI8_dv_TIPI9,TIPI8_dv_TIPI10,TIPI9_dv_TIPI10
0,0,USA,3,553,6,1,1,0,1,1,...,1.000000,1.666667,1.000000,1.666667,1.666667,1.000000,1.666667,0.600000,1.000000,1.666667
1,1,USA,5,85,120,1,1,1,1,1,...,0.400000,2.000000,1.000000,1.000000,5.000000,2.500000,2.500000,0.500000,0.500000,1.000000
2,2,NLD,9,108,100,1,1,0,1,1,...,1.666667,1.250000,1.000000,2.500000,0.750000,0.600000,1.500000,0.800000,2.000000,2.500000
3,3,USA,2,121,139,1,1,0,1,1,...,0.750000,0.750000,1.000000,1.000000,1.000000,1.333333,1.333333,1.333333,1.333333,1.000000
4,4,ITA,3,640,216,1,1,0,1,1,...,1.333333,1.000000,1.333333,2.000000,0.750000,1.000000,1.500000,1.333333,2.000000,1.500000


# Index

In [ ]:
#index drop
index = test['index']
for data in dataset:
    data.drop('index', axis=1, inplace=True)

# Country

In [ ]:
def country_label(data):
    nara = data['country'].copy()
    nara_val = nara.value_counts()
    
    a = [] 
    b = []
    c = [] 
    d = []
    e = []

    
    for i in range(len(nara_val)):
        if nara_val.values[i] <= 10 : a.append(nara_val.index[i])
        elif 100>nara_val.values[i] >10 : b.append(nara_val.index[i])
        elif 500>nara_val.values[i] >= 100 :c.append(nara_val.index[i])
        elif 5000>nara_val.values[i] >= 500 :  d.append(nara_val.index[i])
        elif nara_val.values[i] >= 5000 : e.append(nara_val.index[i])

    for i in range(len(nara)):
        if nara[i] in a: nara[i]=0
        if nara[i] in b: nara[i]=1
        if nara[i] in c: nara[i]=2
        if nara[i] in d: nara[i]=3
        if nara[i] in e: nara[i]=4

        
    nara.fillna(0, inplace = True)
    return nara

In [ ]:
train['nara'] = country_label(train)
test['nara'] = country_label(test)

In [ ]:
train['nara']

0        4
1        4
2        2
3        4
4        2
        ..
14995    4
14996    4
14997    4
14998    4
14999    2
Name: nara, Length: 15000, dtype: int64

In [ ]:
train = train.drop(columns=['country'], axis=1)
test = test.drop(columns=['country'], axis=1)

# VCL

In [ ]:
VCL = []
for i in range(1,17):
    VCL.append('VCL'+str(i))

In [ ]:
#작성자의 영어 실력

# 실제로 있고 진짜 쉬운단어
VCL_1 = ['VCL1','VCL4','VCL5','VCL10','VCL14','VCL16']
# 실제로 있는데 어려운 단어
VCL_2 = ['VCL2','VCL3','VCL7','VCL8','VCL11','VCL13','VCL14']
# 세상에 없는 단어
VCL_3 = ['VCL6','VCL9','VCL12'] # 0 ~ 3 이 나올텐데 3에 가까울 수록 nerdiness일 확률 높다?

In [ ]:
dataset = [train,test]
for data in dataset:
    data['VCL_score'] = data[VCL].sum(axis=1)
    data['VCL_1_score'] = data[VCL_1].sum(axis=1)
    data['VCL_2_score'] = data[VCL_2].sum(axis=1)
    data['VCL_3_score'] = data[VCL_3].sum(axis=1)
    data.drop([('VCL'+str(i) )for i in range(1,17)], axis = 1, inplace = True)

In [ ]:
train.head()

,introelapse,testelapse,surveyelapse,education,urban,gender,engnat,age,hand,religion,...,TIPI7_dv_TIPI9,TIPI7_dv_TIPI10,TIPI8_dv_TIPI9,TIPI8_dv_TIPI10,TIPI9_dv_TIPI10,nara,VCL_score,VCL_1_score,VCL_2_score,VCL_3_score
0,3,553,6,2.0,1,3.0,1.0,20,2.0,12.0,...,1.000000,1.666667,0.600000,1.000000,1.666667,4,8,6,2,0
1,5,85,120,4.0,2,2.0,1.0,49,1.0,2.0,...,2.500000,2.500000,0.500000,0.500000,1.000000,4,11,6,5,0
2,9,108,100,2.0,1,1.0,2.0,43,1.0,2.0,...,0.600000,1.500000,0.800000,2.000000,2.500000,2,11,6,5,0
3,2,121,139,1.0,3,1.0,1.0,17,2.0,1.0,...,1.333333,1.333333,1.333333,1.333333,1.000000,4,9,6,3,0
4,3,640,216,1.0,2,2.0,2.0,18,2.0,12.0,...,1.000000,1.500000,1.333333,2.000000,1.500000,2,8,6,3,0


# Hand

In [ ]:
train.drop('hand', axis=1, inplace = True)
test.drop('hand', axis=1, inplace = True)

# Family size

In [ ]:
#이상치 제거
train = train.drop(train[train.familysize > 50].index)
#test = test.drop(test[test.familysize > 50].index)

# religion & married & voted & orientation

In [ ]:
#train 데이터 결측치 대체: KNN
knull(['familysize','religion','orientation','married','voted','nerdiness'])

# Age

In [ ]:
#이상치 제거
train = train.drop(train[train.age > 100].index)
#test = test.drop(test[test.age > 100].index)

In [ ]:
# age로 연령대 구간 나누기
def age_categorize(age):
    age = (age // 10)
    return age

train['age_cuts'] = age_categorize(train['age'])
test['age_cuts'] = age_categorize(test['age'])

# 기존의 age drop하기
train.drop('age', axis=1, inplace=True)
test.drop('age', axis=1, inplace = True)

# Elapse

In [ ]:
elapse = ['introelapse', 'testelapse', 'surveyelapse']

In [ ]:
train[elapse]=np.log1p(train[elapse])

In [ ]:
#걸린시간 합
train['elapse_sum'] = train['testelapse']+train['surveyelapse']
test['elapse_sum'] = test['testelapse']+test['surveyelapse']

#train.drop('introelapse', axis=1, inplace = True)
#test.drop('introelapse', axis=1, inplace = True)

train.drop('testelapse', axis=1, inplace = True)
test.drop('testelapse', axis=1, inplace = True)

train.drop('surveyelapse', axis=1, inplace = True)
test.drop('surveyelapse', axis=1, inplace = True)

# 나머지 피처 결측치

In [ ]:
#ASD 결측치 최빈값 대체
train['ASD'] = train['ASD'].fillna(train['ASD'].mode()[0])
test['ASD'] = test['ASD'].fillna(train['ASD'].mode()[0])

#education 결측치 최빈값 대체
train['education'] = train['education'].fillna(train['education'].mode()[0])
test['education'] = test['education'].fillna(train['education'].mode()[0])

#engnat 결측치 최빈값 대체
train['engnat'] = train['engnat'].fillna(train['engnat'].mode()[0])
test['engnat'] = test['engnat'].fillna(train['engnat'].mode()[0])

#gender 결측치 최빈값 대체
train['gender'] = train['gender'].fillna(train['gender'].mode()[0])
test['gender'] = test['gender'].fillna(train['gender'].mode()[0])

#religion 결측치 최빈값 대체
test['religion'] = test['religion'].fillna(train['religion'].mode()[0])

#orientation 결측치 최빈값 대체
test['orientation'] = test['orientation'].fillna(train['orientation'].mode()[0])

#voted 결측치 최빈값 대체
test['voted'] = test['voted'].fillna(train['voted'].mode()[0])

#married 결측치 최빈값 대체
test['married'] = test['married'].fillna(train['married'].mode()[0])

#familysize 결측치 최빈값 대체
test['familysize'] = test['familysize'].fillna(train['familysize'].mode()[0])

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
needenco = ['gender', 'religion', 'orientation']
for i in needenco:
    train[i] = encoder.fit_transform(train[i])
    test[i] = encoder.transform(test[i])

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14993 entries, 0 to 14999
Columns: 401 entries, introelapse to elapse_sum
dtypes: float64(391), int64(10)
memory usage: 46.0 MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35452 entries, 0 to 35451
Columns: 400 entries, introelapse to elapse_sum
dtypes: float64(388), int64(12)
memory usage: 108.2 MB


# Model 1: ExtraTreesClassifier

In [ ]:
# features, target split
train_x = train.copy()
train_x.drop('nerdiness', axis=1, inplace = True)
train_y = train['nerdiness']

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(train_x, train_y, test_size=0.25, random_state=42)

In [ ]:
#Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier

X_train, X_test, y_train, y_test = train_test_split(X_train_temp, y_train_temp, test_size=0.25, random_state=42)
xtree = ExtraTreesClassifier(n_estimators = 100, random_state=42)
xtree.fit(X_train, y_train)

preds = xtree.predict(X_test)
auc = roc_auc_score(y_test, preds)
print(auc)

0.7489369825391108


# Model2: ExtraTreesRegressor

In [ ]:
# features, target split
train_x = train.copy()
train_x.drop('nerdiness', axis=1, inplace = True)
train_y = train['nerdiness']

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(train_x, train_y, test_size=0.25, random_state=42)

In [ ]:
#Extra Trees Regressor
from sklearn.ensemble import ExtraTreesRegressor

#X_train, X_test, y_train, y_test = train_test_split(X_train_temp, y_train_temp, test_size=0.25, random_state=42)
xtree_re = ExtraTreesRegressor(n_estimators = 100, random_state=42)
xtree_re.fit(X_train_temp, y_train_temp)

preds = xtree_re.predict(X_test_temp)
auc = roc_auc_score(y_test_temp, preds)
print('Extra Trees Regressor auc: ',auc)

Extra Trees Regressor auc:  0.8706426681708852


# Model3: 4 LGBM Ensemble

In [ ]:
import warnings

warnings.filterwarnings(action='ignore')
#warnings.filterwarnings(action='default')

In [ ]:
# features, target split
train_x = train.copy()
train_x.drop('nerdiness', axis=1, inplace = True)
train_y = train['nerdiness']

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.25, random_state=42)

In [ ]:
def lgbm_rfe_30(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=30)
        model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'score':score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats*ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

lgbm_30 = lgbm_rfe_30(X_train, y_train)

400 0.838650413316031
360 0.8327561895225152
324 0.8325329273401657
291 0.8370599895776921
261 0.8324823484933633
234 0.8396221402515658
210 0.8356815882779691
189 0.8318467920749589
170 0.832510447852698
153 0.8354108126334719
137 0.836949124832681
123 0.8433700838893601
110 0.8401994543615314
99 0.835634585713264
89 0.838358179978951
80 0.8380301838208998
72 0.8349908549357801
64 0.8299242849990293
57 0.8336660978675141
51 0.8306640644956932
45 0.8305307202632143
40 0.8338392921005039


In [ ]:
model1 = LGBMClassifier(objective="binary", num_iterations=10**3)
x_train_1 = X_train[lgbm_30.iloc[7,2]]
model1.fit(x_train_1, y_train)

pred_y1 = model1.predict_proba(X_test[lgbm_30.iloc[7,2]])
pred_y1 = pred_y1[:,1]

In [ ]:
def lgbm_rfe_500(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=500)
        model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'score':score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats*ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

lgbm_500 = lgbm_rfe_500(X_train, y_train)

400 0.8351892844698601
360 0.8314251163172026
324 0.8381991842423498
291 0.8358979483080203
261 0.8330730287564819
234 0.833065855008301
210 0.8324642849822705
189 0.8310013527639427
170 0.8349002849002849
153 0.8342218532866015
137 0.8290444567422985
123 0.8358605423353624
110 0.8318811617372769
99 0.8320661419582283
89 0.8327394494660681
80 0.8300354588124372
72 0.8292494209760397
64 0.8304207915718709
57 0.8231717190350284
51 0.8259341244952756
45 0.8217374818094243
40 0.8234494455717477


In [ ]:
model2 = LGBMClassifier(objective="binary", num_iterations=10**3)
x_train_2 = X_train[lgbm_500.iloc[7,2]]
model2.fit(x_train_2, y_train)

pred_y2 = model2.predict_proba(X_test[lgbm_500.iloc[7,2]])
pred_y2 = pred_y2[:,1]

In [ ]:
def lgbm_rfe_913(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=913)
        model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'score':score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats*ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

lgbm_913 = lgbm_rfe_913(X_train, y_train)

400 0.8264579677722361
360 0.8296508334536932
324 0.8273686054337104
291 0.82381824532722
261 0.823754736288962
234 0.8247237612920606
210 0.8218709762670822
189 0.8234054365785428
170 0.8252871479217692
153 0.8274187980607207
137 0.8283038272390266
123 0.8258884351065672
110 0.8271698836043224
99 0.8222632982292245
89 0.8262126385034811
80 0.8230474299838463
72 0.8294326491770969
64 0.8241583259836988
57 0.8194151227312165
51 0.8112408949038095
45 0.8196041133370005
40 0.8228046615634081


In [ ]:
model3 = LGBMClassifier(objective="binary", num_iterations=10**3)
x_train_3 = X_train[lgbm_913.iloc[7,2]]
model3.fit(x_train_3, y_train)

pred_y3 = model3.predict_proba(X_test[lgbm_913.iloc[7,2]])
pred_y3 = pred_y3[:,1]

In [ ]:
def lgbm_rfe_8(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=8)
        model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'score':score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats*ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

lgbm_8 = lgbm_rfe_8(X_train, y_train)

400 0.8164896491668145
360 0.8268720926201241
324 0.8253976265787288
291 0.81930347284678
261 0.8200055266984401
234 0.8289405257909195
210 0.8218888868495168
189 0.8205276059606768
170 0.8235530144978964
153 0.8241495308424441
137 0.82136680758728
123 0.8210700789440947
110 0.8241459619412375
99 0.8229167303970454
89 0.8170514961853546
80 0.8279636665463437
72 0.8209171260352363
64 0.8190036851454174
57 0.8196904437061917
51 0.8156642133020086
45 0.8114692248550516
40 0.8215855302469475


In [ ]:
model4 = LGBMClassifier(objective="binary", num_iterations=10**3)
x_train_4 = X_train[lgbm_8.iloc[7,2]]
model4.fit(x_train_4, y_train)

pred_y4 = model4.predict_proba(X_test[lgbm_8.iloc[7,2]])
pred_y4 = pred_y4[:,1]

In [ ]:
pred_all = (pred_y1 + pred_y2 + pred_y3 + pred_y4) * (1/4)

lgbm_ensemble_nerdiness = pd.DataFrame({
    'nerdiness':pred_all
})

In [ ]:
auc = roc_auc_score(y_test, pred_all)
print('LGBM Ensemble auc: ',auc)

LGBM Ensemble auc:  0.8674108408386134


# Model 2 + 3

In [ ]:
pred_y_ensemble = (preds)*(0.7) + (pred_all)*(0.3)
auc = roc_auc_score(y_test, pred_y_ensemble)
print('combined_2_3 auc: ', auc)

combined_2_3 auc:  0.8752872879158236


# Model 2-2: Extra Trees Regressor 교차검증

In [ ]:
# features, target split
train_x = train.copy()
train_x.drop('nerdiness', axis=1, inplace = True)
train_y = train['nerdiness']

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(train_x, train_y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_estimators=1000, random_state=42)
scores = cross_validate(et, X_train_temp, y_train_temp, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

1.0 0.7690329351597942


# ExtraTreesRegressor 하이퍼파라미터 조정

In [ ]:
# features, target split
train_x = train.copy()
train_x.drop('nerdiness', axis=1, inplace = True)
train_y = train['nerdiness']

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(train_x, train_y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_estimators=1000, random_state=42)
scores = cross_validate(et, X_train_temp, y_train_temp, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# ExtraTreesRegressor 하이퍼파라미터 튜닝 시도

In [ ]:
# features, target split
train_x = train.copy()
train_x.drop('nerdiness', axis=1, inplace = True)
train_y = train['nerdiness']

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(train_x, train_y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV

model_extra_re = ExtraTreesRegressor(n_estimators=100, min_samples_split = 25, min_samples_leaf = 35, max_features=150)

gsc = GridSearchCV(
    estimator=model_extra_re,
    param_grid = {
        'n_estimators' : range(50, 126, 25),
        'max_features' : range(50, 401, 50),
        'min_samples_leaf' : range(20, 50, 5),
        'min_samples_split' : range(15, 36, 5)
    },
    scoring='r2',
    cv = 5
)


grid_result = gsc.fit(X_train_temp, y_train_temp)

print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))

model_extra_re_final = ExtraTreesRegressor(**grid_result.best_params_)
model_extra_re_final.fit(X_train_temp, y_train_temp)

predict = model_extra_re_final.predict(X_test_temp)
auc_score = roc_auc_score(predict, y_test_temp)

KeyboardInterrupt: 

In [ ]:
print('GridSearch Tuning: ',auc_score)